# TTRSS-Notion 동기화 애플리케이션 테스트

이 노트북은 TTRSS와 Notion 데이터베이스 간 동기화 기능을 테스트합니다. 모듈별로 주요 기능을 테스트하고 결과를 확인합니다.

## 1. 환경 설정 및 임포트

In [1]:
import sys
import os
from datetime import datetime
import json

# 현재 경로를 확인합니다
print(f"Current working directory: {os.getcwd()}")

# 모듈 임포트를 위해 필요한 경우 경로를 추가합니다
if not os.getcwd().endswith('ttrss_notion_sync_app'):
    os.chdir('ttrss_notion_sync_app')
    print(f"Changed directory to: {os.getcwd()}")

Current working directory: /Users/oldman/Library/CloudStorage/OneDrive-개인/001_Documents/001_TelePIX/000_workspace/ttrss_notion_sync_app


In [2]:
# 필요한 모듈을 임포트합니다
from dotenv import load_dotenv
load_dotenv()  # .env 파일에서 환경 변수를 로드합니다

# 설정 가져오기
from config import DB_CONFIG, NOTION_DATABASE_ID, NOTION_API_KEY, logger

# 설정 정보 확인 (API 키 일부만 표시)
print(f"Database Config: {DB_CONFIG['host']}:{DB_CONFIG['port']} - DB: {DB_CONFIG['dbname']}")
print(f"Notion Database ID: {NOTION_DATABASE_ID}")
print(f"Notion API Key: {NOTION_API_KEY[:5]}...{NOTION_API_KEY[-5:]}")

Database Config: 222.110.59.179:5432 - DB: ttrss
Notion Database ID: 1acf199edf7c8061a721d4b4d2dc3cb4
Notion API Key: ntn_6...rVaAa


## 2. 데이터베이스 관리자 테스트

데이터베이스 연결 및 기본 함수들을 테스트합니다.

In [3]:
from db_manager import DatabaseManager

# 데이터베이스 관리자 객체 생성
db_manager = DatabaseManager()

# 데이터베이스 연결
try:
    conn = db_manager.connect()
    print("✅ 데이터베이스 연결 성공")
except Exception as e:
    print(f"❌ 데이터베이스 연결 실패: {e}")

2025-03-23 18:40:30,163 - INFO - Connected to TTRSS database successfully


✅ 데이터베이스 연결 성공


In [4]:
# TTRSS 항목 테이블 스키마 조회
try:
    schema = db_manager.get_ttrss_entries_schema()
    print("✅ TTRSS 항목 테이블 스키마 조회 성공")
    print(f"스키마 컬럼 수: {len(schema)}")
    for column, data_type in schema[:10]:  # 처음 10개 컬럼만 출력
        print(f"  - {column}: {data_type}")
except Exception as e:
    print(f"❌ TTRSS 항목 테이블 스키마 조회 실패: {e}")

2025-03-23 18:40:30,966 - INFO - Retrieved ttrss_entries schema


✅ TTRSS 항목 테이블 스키마 조회 성공
스키마 컬럼 수: 17
  - id: integer
  - title: text
  - guid: text
  - link: text
  - updated: timestamp without time zone
  - content: text
  - content_hash: character varying
  - cached_content: text
  - no_orig_date: boolean
  - date_entered: timestamp without time zone


In [5]:
# TTRSS 항목 조회
try:
    entries = db_manager.get_ttrss_entries()
    print(f"✅ TTRSS 항목 조회 성공: {len(entries)}개 항목 조회됨")
    
    # 첫 5개 항목 미리보기
    if entries:
        print("\n처음 5개 항목:")
        for entry in entries[:5]:
            print(f"  - ID: {entry['id']}, 제목: {entry['title'][:50]}...")
    else:
        print("TTRSS 항목이 없습니다. 피드를 구독하고 업데이트해야 할 수 있습니다.")
except Exception as e:
    print(f"❌ TTRSS 항목 조회 실패: {e}")

2025-03-23 18:40:31,742 - INFO - Retrieved 38 entries from ttrss_entries


✅ TTRSS 항목 조회 성공: 38개 항목 조회됨

처음 5개 항목:
  - ID: 37, 제목: 👏👏 Here are our 9 confirmed speakers for our miniC...
  - ID: 30, 제목: Code Implementation of a Rapid Disaster Assessment...
  - ID: 4, 제목: Microsoft AI Introduces Claimify: A Novel LLM-base...
  - ID: 12, 제목: Groundlight Research Team Released an Open-Source ...
  - ID: 13, 제목: Cohere Released Command A: A 111B Parameter AI Mod...


In [6]:
# 동기화 테이블 생성 테스트
try:
    was_created = db_manager.create_sync_table()
    if was_created:
        print("✅ 동기화 테이블이 생성되었습니다.")
    else:
        print("ℹ️ 동기화 테이블이 이미 존재합니다. 테이블을 유지합니다.")
except Exception as e:
    print(f"❌ 동기화 테이블 작업 실패: {e}")

2025-03-23 18:40:32,028 - INFO - Table ttrss_notion_sync already exists, skipping creation


ℹ️ 동기화 테이블이 이미 존재합니다. 테이블을 유지합니다.


In [7]:
# 동기화 테이블 항목 조회
try:
    sync_entries = db_manager.get_sync_entries()
    print(f"✅ 동기화 테이블 항목 조회 성공: {len(sync_entries)}개 항목 조회됨")
    
    # 항목이 있으면 첫 5개 항목 미리보기
    if sync_entries:
        print("\n처음 5개 항목:")
        for entry in sync_entries[:5]:
            print(f"  - ID: {entry['id']}, 제목: {entry['title'][:50] if entry['title'] else 'No Title'}...")
    else:
        print("동기화 테이블에 항목이 없습니다. 아직 동기화가 진행되지 않았을 수 있습니다.")
except Exception as e:
    print(f"❌ 동기화 테이블 항목 조회 실패: {e}")

2025-03-23 18:40:32,510 - INFO - Retrieved 38 entries from ttrss_notion_sync


✅ 동기화 테이블 항목 조회 성공: 38개 항목 조회됨

처음 5개 항목:
  - ID: 13, 제목: A Coding Guide to Build an Optical Character Recog...
  - ID: 4, 제목: A Coding Implementation to Build a Document Search...
  - ID: 5, 제목: Kyutai Releases MoshiVis: The First Open-Source Re...
  - ID: 6, 제목: Meet Attentive Reasoning Queries (ARQs): A Structu...
  - ID: 9, 제목: A Step-by-Step Guide to Building a Semantic Search...


## 3. Notion 관리자 테스트

Notion API 연결 및 기본 함수들을 테스트합니다.

In [8]:
from notion_manager import NotionManager

# Notion 관리자 객체 생성
notion_manager = NotionManager()

# Notion API 연결
try:
    client = notion_manager.connect()
    print("✅ Notion API 연결 성공")
except Exception as e:
    print(f"❌ Notion API 연결 실패: {e}")

2025-03-23 18:40:34,169 - INFO - HTTP Request: GET https://api.notion.com/v1/databases/1acf199edf7c8061a721d4b4d2dc3cb4 "HTTP/1.1 200 OK"
2025-03-23 18:40:34,172 - INFO - Connected to Notion database: 읽을거리


✅ Notion API 연결 성공


In [9]:
# Notion 데이터베이스 구조 조회
try:
    properties = notion_manager.get_database_structure()
    print("✅ Notion 데이터베이스 구조 조회 성공")
    print(f"속성 수: {len(properties)}")
    print("데이터베이스 속성:")
    for prop_name, prop_data in properties.items():
        print(f"  - {prop_name} ({prop_data.get('type', 'unknown')})")
except Exception as e:
    print(f"❌ Notion 데이터베이스 구조 조회 실패: {e}")

2025-03-23 18:40:36,861 - INFO - HTTP Request: GET https://api.notion.com/v1/databases/1acf199edf7c8061a721d4b4d2dc3cb4 "HTTP/1.1 200 OK"
2025-03-23 18:40:36,864 - INFO - Retrieved Notion database structure


✅ Notion 데이터베이스 구조 조회 성공
속성 수: 12
데이터베이스 속성:
  - Why it matters (rich_text)
  - 담당 (people)
  - Read (checkbox)
  - URL (url)
  - Score (select)
  - Status (select)
  - 리뷰어 코멘트 (rich_text)
  - AI summary (rich_text)
  - Created (created_time)
  - 핸드북의 카테고리 (multi_select)
  - Type (select)
  - Name (title)


In [10]:
# Notion 데이터베이스 페이지 조회
try:
    pages = notion_manager.get_database_pages()
    print(f"✅ Notion 데이터베이스 페이지 조회 성공: {len(pages)}개 페이지 조회됨")
    
    # 첫 3개 페이지 간략 정보
    if pages:
        print("\n처음 3개 페이지 ID:")
        for i, page in enumerate(pages[:3]):
            page_id = page.get('id', 'Unknown ID')
            print(f"  - Page {i+1}: {page_id}")
            
        # 첫 번째 페이지의 데이터 추출 테스트
        if pages:
            page_data = notion_manager.extract_page_data(pages[0])
            print(f"\n첫 번째 페이지 데이터 추출:")
            for key, value in page_data.items():
                if key == 'content' and value:
                    print(f"  - {key}: {value[:100]}...")
                else:
                    print(f"  - {key}: {value}")
    else:
        print("Notion 데이터베이스에 페이지가 없습니다.")
except Exception as e:
    print(f"❌ Notion 데이터베이스 페이지 조회 실패: {e}")

2025-03-23 18:40:38,065 - INFO - HTTP Request: POST https://api.notion.com/v1/databases/1acf199edf7c8061a721d4b4d2dc3cb4/query "HTTP/1.1 200 OK"
2025-03-23 18:40:38,074 - INFO - Retrieved 47 pages from Notion database


✅ Notion 데이터베이스 페이지 조회 성공: 47개 페이지 조회됨

처음 3개 페이지 ID:
  - Page 1: 1bff199e-df7c-8048-9010-dd8ead595457
  - Page 2: 1bff199e-df7c-8018-a41e-fb57d514ea86
  - Page 3: 1bff199e-df7c-80ae-9799-f23d4929f467

첫 번째 페이지 데이터 추출:
  - notion_page_id: 1bff199e-df7c-8048-9010-dd8ead595457
  - title: The Ultimate Rules Template for CLINE/Cursor/RooCode/Windsurf that Actually Makes AI Remember Everything! (w/ Memory Bank & Software Engineering Best Practices)
  - link: None
  - content: 
  - updated: 2025-03-23 05:59:00+00:00
  - published: 2025-03-23 05:59:00+00:00


## 4. 동기화 관리자 테스트

데이터베이스와 Notion 간 동기화 기능을 테스트합니다.

In [11]:
from sync_manager import SyncManager

# 동기화 관리자 객체 생성
sync_manager = SyncManager()

# 초기화 (Notion 연결 없이)
try:
    sync_manager.initialize(connect_notion=False) # notion에 연결하지 않고 테스트할 수 있습니다.
    print("✅ 동기화 관리자 초기화 성공 (Notion 연결 없음)")
except Exception as e:
    print(f"❌ 동기화 관리자 초기화 실패: {e}")

2025-03-23 18:40:39,521 - INFO - Connected to TTRSS database successfully
2025-03-23 18:40:39,522 - INFO - Database connection established
2025-03-23 18:40:39,522 - INFO - Skipping Notion API connection as requested
2025-03-23 18:40:39,547 - INFO - Table ttrss_notion_sync already exists, skipping creation
2025-03-23 18:40:39,547 - INFO - Sync table already exists
2025-03-23 18:40:39,548 - INFO - Initialization complete


✅ 동기화 관리자 초기화 성공 (Notion 연결 없음)


In [12]:
# 초기화 (Notion 연결 없이)
try:
    sync_manager.initialize(connect_notion=True)
    print("✅ 동기화 관리자 초기화 성공 (Notion 연결 있음)")
except Exception as e:
    print(f"❌ 동기화 관리자 초기화 실패: {e}")

# Notion 데이터베이스 확인 (이제 자동으로 연결됨)
try:
    notion_pages = sync_manager.check_notion_database()
    print(f"✅ Notion 데이터베이스 확인 성공: {len(notion_pages)}개 페이지 조회됨")
    
    # 첫 3개 페이지 정보
    if notion_pages:
        print("\n처음 3개 페이지:")
        for i, page in enumerate(notion_pages[:3]):
            print(f"  - Page {i+1}: {page['title'][:50]}... (ID: {page['notion_page_id'][:10]}...)")
    else:
        print("Notion 데이터베이스에 페이지가 없습니다.")
except Exception as e:
    print(f"❌ Notion 데이터베이스 확인 실패: {e}")

2025-03-23 18:40:40,912 - INFO - Connected to TTRSS database successfully
2025-03-23 18:40:40,912 - INFO - Database connection established
2025-03-23 18:40:41,363 - INFO - HTTP Request: GET https://api.notion.com/v1/databases/1acf199edf7c8061a721d4b4d2dc3cb4 "HTTP/1.1 200 OK"
2025-03-23 18:40:41,365 - INFO - Connected to Notion database: 읽을거리
2025-03-23 18:40:41,366 - INFO - Notion API connection established
2025-03-23 18:40:41,391 - INFO - Table ttrss_notion_sync already exists, skipping creation
2025-03-23 18:40:41,391 - INFO - Sync table already exists
2025-03-23 18:40:41,392 - INFO - Initialization complete


✅ 동기화 관리자 초기화 성공 (Notion 연결 있음)


2025-03-23 18:40:42,055 - INFO - HTTP Request: POST https://api.notion.com/v1/databases/1acf199edf7c8061a721d4b4d2dc3cb4/query "HTTP/1.1 200 OK"
2025-03-23 18:40:42,067 - INFO - Retrieved 47 pages from Notion database
Processing Notion pages: 100%|██████████| 47/47 [00:00<00:00, 136329.38it/s]
2025-03-23 18:40:42,092 - INFO - Processed 47 pages from Notion database


✅ Notion 데이터베이스 확인 성공: 47개 페이지 조회됨

처음 3개 페이지:
  - Page 1: The Ultimate Rules Template for CLINE/Cursor/RooCo... (ID: 1bff199e-d...)
  - Page 2: AI Mock Interview System... (ID: 1bff199e-d...)
  - Page 3: GPT - free tool to navigate to any section of chat... (ID: 1bff199e-d...)


In [13]:
# TTRSS 항목 확인
try:
    ttrss_entries = sync_manager.check_ttrss_entries()
    print(f"✅ TTRSS 항목 확인 성공: {len(ttrss_entries)}개 항목 조회됨")
    
    # 첫 3개 항목 정보
    if ttrss_entries:
        print("\n처음 3개 항목:")
        for i, entry in enumerate(ttrss_entries[:3]):
            print(f"  - Entry {i+1}: {entry['title'][:50]}... (ID: {entry['id']})")
    else:
        print("TTRSS에 항목이 없습니다. 피드를 구독하고 업데이트해야 할 수 있습니다.")
except Exception as e:
    print(f"❌ TTRSS 항목 확인 실패: {e}")

2025-03-23 18:40:43,601 - INFO - Retrieved 38 entries from ttrss_entries


✅ TTRSS 항목 확인 성공: 38개 항목 조회됨

처음 3개 항목:
  - Entry 1: 👏👏 Here are our 9 confirmed speakers for our miniC... (ID: 37)
  - Entry 2: Code Implementation of a Rapid Disaster Assessment... (ID: 30)
  - Entry 3: Microsoft AI Introduces Claimify: A Novel LLM-base... (ID: 4)


In [14]:
# TTRSS와 동기화 테이블 비교
try:
    new_entries = sync_manager.compare_ttrss_with_sync()
    print(f"✅ TTRSS와 동기화 테이블 비교 성공: {len(new_entries)}개의 새 항목 발견")
    
    # 첫 3개 새 항목 정보
    if new_entries:
        print("\n처음 3개 새 항목:")
        for i, entry in enumerate(new_entries[:3]):
            print(f"  - New Entry {i+1}: {entry['title'][:50]}... (ID: {entry['id']})")
    else:
        print("모든 TTRSS 항목이 이미 동기화 테이블에 있습니다.")
except Exception as e:
    print(f"❌ TTRSS와 동기화 테이블 비교 실패: {e}")

2025-03-23 18:40:45,674 - INFO - Retrieved 38 entries from ttrss_entries
2025-03-23 18:40:45,686 - INFO - Found 0 entries in TTRSS that are not in sync table
2025-03-23 18:40:45,686 - INFO - Found 0 new entries in TTRSS


✅ TTRSS와 동기화 테이블 비교 성공: 0개의 새 항목 발견
모든 TTRSS 항목이 이미 동기화 테이블에 있습니다.


In [15]:
# Notion과 동기화 테이블 비교
try:
    new_in_notion, missing_from_notion = sync_manager.compare_notion_with_sync()
    print(f"✅ Notion과 동기화 테이블 비교 성공")
    print(f"  - Notion에 새로 추가된 항목: {len(new_in_notion)}개")
    print(f"  - Notion에서 삭제된 항목: {len(missing_from_notion)}개")
    
    # 새로 추가된 항목 정보
    if new_in_notion:
        print("\n처음 3개 새로 추가된 Notion 항목:")
        for i, page in enumerate(new_in_notion[:3]):
            print(f"  - New Page {i+1}: {page['title'][:50]}... (ID: {page['notion_page_id'][:10]}...)")
    
    # 삭제된 항목 정보
    if missing_from_notion:
        print("\n처음 3개 삭제된 Notion 항목:")
        for i, entry in enumerate(missing_from_notion[:3]):
            print(f"  - Missing Page {i+1}: {entry['title'][:50] if entry['title'] else 'No Title'}... (ID: {entry['notion_page_id'][:10]}...)")
except Exception as e:
    print(f"❌ Notion과 동기화 테이블 비교 실패: {e}")

2025-03-23 18:40:55,600 - INFO - HTTP Request: POST https://api.notion.com/v1/databases/1acf199edf7c8061a721d4b4d2dc3cb4/query "HTTP/1.1 200 OK"
2025-03-23 18:40:55,608 - INFO - Retrieved 47 pages from Notion database
Processing Notion pages: 100%|██████████| 47/47 [00:00<00:00, 431361.68it/s]
2025-03-23 18:40:55,609 - INFO - Processed 47 pages from Notion database
2025-03-23 18:40:55,647 - INFO - Retrieved 38 entries from ttrss_notion_sync
2025-03-23 18:40:55,648 - INFO - Found 47 new pages in Notion
2025-03-23 18:40:55,648 - INFO - Found 0 sync entries missing from Notion


✅ Notion과 동기화 테이블 비교 성공
  - Notion에 새로 추가된 항목: 47개
  - Notion에서 삭제된 항목: 0개

처음 3개 새로 추가된 Notion 항목:
  - New Page 1: The Ultimate Rules Template for CLINE/Cursor/RooCo... (ID: 1bff199e-d...)
  - New Page 2: AI Mock Interview System... (ID: 1bff199e-d...)
  - New Page 3: GPT - free tool to navigate to any section of chat... (ID: 1bff199e-d...)


## 5. 샘플 동기화 실행 (테스트 전용)

⚠️ 주의: 이 섹션은 실제 동기화를 수행합니다. 테스트 환경에서만 실행하세요.

In [16]:
# TTRSS 항목을 동기화 테이블로 동기화
# 실행하기 전에 주석 처리를 제거하세요

# try:
#     added = sync_manager.sync_ttrss_to_sync_table()
#     print(f"✅ TTRSS 항목 동기화 성공: {added}개 항목 추가됨")
# except Exception as e:
#     print(f"❌ TTRSS 항목 동기화 실패: {e}")

2025-03-23 18:41:17,128 - INFO - Retrieved 38 entries from ttrss_entries
2025-03-23 18:41:17,137 - INFO - Found 0 entries in TTRSS that are not in sync table
2025-03-23 18:41:17,138 - INFO - Found 0 new entries in TTRSS
Adding new TTRSS entries to sync table: 0it [00:00, ?it/s]
2025-03-23 18:41:17,139 - INFO - Added 0 new TTRSS entries to sync table


✅ TTRSS 항목 동기화 성공: 0개 항목 추가됨


In [19]:
# Notion 데이터베이스를 동기화 테이블로 동기화
# 실행하기 전에 주석 처리를 제거하세요

# try:
#     added, removed = sync_manager.sync_notion_to_sync_table()
#     print(f"✅ Notion 데이터베이스 동기화 성공")
#     print(f"  - 추가된 항목: {added}개")
#     print(f"  - 제거된 항목: {removed}개")
# except Exception as e:
#     print(f"❌ Notion 데이터베이스 동기화 실패: {e}")

2025-03-23 18:41:57,085 - INFO - HTTP Request: POST https://api.notion.com/v1/databases/1acf199edf7c8061a721d4b4d2dc3cb4/query "HTTP/1.1 200 OK"
2025-03-23 18:41:57,246 - INFO - Retrieved 47 pages from Notion database
Processing Notion pages: 100%|██████████| 47/47 [00:00<00:00, 228162.37it/s]
2025-03-23 18:41:57,248 - INFO - Processed 47 pages from Notion database
2025-03-23 18:41:57,286 - INFO - Retrieved 79 entries from ttrss_notion_sync
2025-03-23 18:41:57,287 - INFO - Found 6 new pages in Notion
2025-03-23 18:41:57,287 - INFO - Found 0 sync entries missing from Notion
Adding new Notion pages to sync table:   0%|          | 0/6 [00:00<?, ?it/s]2025-03-23 18:41:57,296 - INFO - Found duplicate entry: 36 (source: ttrss)
2025-03-23 18:41:57,296 - INFO - Skipping duplicate Notion entry: 1bff199e-df7c-8008-a051-d277f5ca2d34 (title: [ICASSP 2025] BANC: Towards Efficient Binaural Audio Neural Codec for Overlapping Speech)
2025-03-23 18:41:57,304 - INFO - Found duplicate entry: 16 (source: 

✅ Notion 데이터베이스 동기화 성공
  - 추가된 항목: 6개
  - 제거된 항목: 0개


In [20]:
# 동기화 테이블의 항목을 Notion으로 동기화
# 실행하기 전에 주석 처리를 제거하세요

try:
    synced = sync_manager.sync_to_notion()
    print(f"✅ Notion으로 동기화 성공: {synced}개 항목 동기화됨")
except Exception as e:
    print(f"❌ Notion으로 동기화 실패: {e}")

2025-03-23 18:44:35,981 - INFO - Found 21 entries to sync to Notion
Syncing to Notion:   0%|          | 0/21 [00:00<?, ?it/s]2025-03-23 18:44:36,810 - INFO - HTTP Request: POST https://api.notion.com/v1/pages "HTTP/1.1 400 Bad Request"
2025-03-23 18:44:36,812 - ERROR - Error creating Notion page: body failed validation: body.properties.Why it matters.rich_text[0].text.content.length should be ≤ `2000`, instead was `2002`.
2025-03-23 18:44:36,813 - ERROR - Entry data: {"id": 9, "ttrss_entry_id": 3, "title": "A Step-by-Step Guide to Building a Semantic Search Engine with Sentence Transformers, FAISS, and all-MiniLM-L6-v2 [\ud83d\udcbb Colab Notebook Included]", "content": "<table> <tr><td> <a href=\"https://www.reddit.com/r/machinelearningnews/comments/1jg8egg/a_stepbystep_guide_to_building_a_semantic_search/\"> <img src=\"https://b.thumbs.redditmedia.com/uPep74DW943R4jFYESjPRZ-4icpFGNkFPpP8pOpuLeU.jpg\" alt=\"A Step-by-Step Guide to Building a Semantic Search...
Syncing to Notion:   5%|

✅ Notion으로 동기화 성공: 20개 항목 동기화됨


In [21]:
# 전체 동기화 실행
# 실행하기 전에 주석 처리를 제거하세요

try:
    results = sync_manager.perform_full_sync()
    print("✅ 전체 동기화 성공")
    print(f"  - Notion에서 추가된 항목: {results['added_from_notion']}개")
    print(f"  - 동기화 테이블에서 제거된 항목: {results['removed_from_sync']}개")
    print(f"  - TTRSS에서 추가된 항목: {results['added_from_ttrss']}개")
    print(f"  - Notion으로 동기화된 항목: {results['synced_to_notion']}개")
except Exception as e:
    print(f"❌ 전체 동기화 실패: {e}")

2025-03-23 18:49:49,064 - INFO - Step 1: Syncing Notion to sync table
2025-03-23 18:49:51,450 - INFO - HTTP Request: POST https://api.notion.com/v1/databases/1acf199edf7c8061a721d4b4d2dc3cb4/query "HTTP/1.1 200 OK"
2025-03-23 18:49:51,460 - INFO - Retrieved 67 pages from Notion database
Processing Notion pages: 100%|██████████| 67/67 [00:00<00:00, 462962.71it/s]
2025-03-23 18:49:51,461 - INFO - Processed 67 pages from Notion database
2025-03-23 18:49:51,502 - INFO - Retrieved 79 entries from ttrss_notion_sync
2025-03-23 18:49:51,503 - INFO - Found 6 new pages in Notion
2025-03-23 18:49:51,503 - INFO - Found 0 sync entries missing from Notion
Adding new Notion pages to sync table:   0%|          | 0/6 [00:00<?, ?it/s]2025-03-23 18:49:51,511 - INFO - Found duplicate entry: 36 (source: ttrss)
2025-03-23 18:49:51,511 - INFO - Skipping duplicate Notion entry: 1bff199e-df7c-8008-a051-d277f5ca2d34 (title: [ICASSP 2025] BANC: Towards Efficient Binaural Audio Neural Codec for Overlapping Speech

✅ 전체 동기화 성공
  - Notion에서 추가된 항목: 6개
  - 동기화 테이블에서 제거된 항목: 0개
  - TTRSS에서 추가된 항목: 0개
  - Notion으로 동기화된 항목: 0개


## 6. 연결 종료

In [22]:
# 연결 종료
sync_manager.close()
print("✅ 모든 연결 종료 완료")

2025-03-23 18:50:09,026 - INFO - Database connection closed
2025-03-23 18:50:09,026 - INFO - Connections closed


✅ 모든 연결 종료 완료


## 7. 테스트 정리

테스트 결과를 정리합니다. 각 테스트의 성공/실패 여부와 조치가 필요한 항목을 확인하세요.

### 다음 단계

1. 실제 동기화를 수행하려면 5번 섹션의 주석을 제거하고 실행하세요.
2. 명령줄에서 동기화를 실행하려면 `python -m ttrss_notion_sync_app.main full-sync` 명령을 사용하세요.
3. 정기적인 동기화를 위해 cron 작업이나 스케줄러를 설정하는 것을 고려하세요.